# DAS Microseismic Detection

### Written by Paige Given and Fantine Huot with Contributions from Ariel Lellouch, Bin Luo, Robert G. Clapp, Tamas Nemeth, Kurt Nihei, and Biondo L. Biondi

#### This notebook runs through the entire processing and prediction process for our ML workflow. 

## Load dependencies

In [ ]:
 cd ..

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import scipy
import math
from obspy.signal.filter import bandpass
import matplotlib.pyplot as plt
import enum
import os
from typing import List, Sequence, Text, Tuple
import logging
import os

## Check for visible GPU devices
#### To maintain efficiency when working with large data, you will want to ensure you have a GPU connected

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.test.gpu_device_name()

In [ ]:
!nvidia-smi

## Set Datapath and Load in Data

#### Set datapath and file pattern for unprocessed data that you want to run predictions on:

In [ ]:
datapath = '/scratch/users/prgiven/continuous_data/continuous_processed/'# replace me with datapath 
#input_file_pattern = os.path.join(datapath, 'unprocessed_data/2hr/', '*.npy') # replace me with directory and file pattern       
input_file_pattern = os.path.join(datapath, 'unprocessed_data/day7/','*a*.npy') # replace me with directory and file pattern           

#### Find filename and load data from numpy file

In [ ]:
filenames = sorted(tf.io.gfile.glob(input_file_pattern))
print('Found {} files'.format(len(filenames)))
print(filenames)

In [ ]:
print(filenames[0])
data=np.load(filenames[0]) 

In [ ]:
#use this if data in segy format
#import segyio
#with segyio.open('/scratch/users/prgiven/FORGE/FORGE_78-32_iDASv3-P11_UTC190427171923.sgy', ignore_geometry=True) as f:
#    FORGE = segyio.tools.collect(f.trace[:])
#print(FORGE.shape)

#### Quality check by printing data and checking data shape

In [ ]:
print(data)

In [ ]:
data.shape

## Data pre-processing functions

**bp filter**: band-pass filters the data.\
**run prediction**: runs the model prediction on the data\
**preprocess**: runs bandpass on data, reshapes to model input size, clips amplitude values, and converts to float 32 format

In [ ]:
def bpfilter(data,dt,bp_low,bp_high):
    sos = scipy.signal.iirfilter(N=15,Wn=[bp_low,bp_high],btype='bandpass',fs=1/dt,output='sos')
    return np.float32(scipy.signal.sosfilt(sos,data,axis=-1))

def normalize(data, axis=-1):
    stddev = np.std(data, axis=axis, keepdims=True)
    return np.divide(data, stddev, out=np.zeros_like(data), where=stddev != 0)

def med(data,axis=0):
    data = data - np.median(data, axis=axis)
    return data

In [ ]:
def run_prediction(inputs, model):
    return model.predict(inputs, use_multiprocessing=True)

def preprocess(data):
    # crop it to input_height and width (cropping at the center)
    data=med(data,axis=0)
    data = bpfilter(data,0.002,30,200) #can change 
    data=normalize(data,axis=-1)
    data = data.reshape((1, 400, 250, 1)) # model input shape
    data = np.clip(data, -10, 10) # stats
    return np.float32(data)


In [ ]:
#cut data noise if necessary 
data=data[100:3968,0:7260000]

In [ ]:
#double check data shape 
data.shape

## Load ML Model and Compile

In [ ]:
job_id = 'train_20210525_142709_cnn2d_modular_dec_baseline_' # Replace me
datapath2='/home/users/prgiven/DAS_ML/microseismic-detection-ml'
ckpt = '{}/models/{}/ckpt'.format(datapath2, job_id)
model = tf.keras.models.load_model(ckpt)
model.compile()

## Implement Sliding Windows, Pre-Procesing, and Running Prediction on Data

The cell below will create sliding windows from continuous data with window size (400,250). It the pre-processes the data, and runs the prediction on the processed windows. The final output is in the form [inputted,logits] where the inputted is the inputted window, and the logits is the result of the predictions.

In [ ]:
#data=np.load(filenames[0])
def sliding_window(data):
    n1=data.shape[0]
    w1=400
    d1=395 #sliding window with overlap of 5 cells
    n2=data.shape[1]
    w2=250
    d2=245 #sliding window with overlap of 5 cells
    #data_new=[]
    num_windows1=len(range(0,n1-w1+1,d1))
    #print(num_windows1)
    num_windows2=len(range(0,n2-w2+1,d2))
    #print(num_windows2)
    logits=np.zeros((len(range(0,n1-w1+1,d1)),len(range(0,n2-w2+1,d2))))
    inputted=np.zeros((num_windows1,num_windows2,400,250))
    count1=0
    for i1 in range(0,n1-w1+1,d1):
        count2=0
        print(count2)
        for i2 in range(0,n2-w2+1,d2):
            #print(i2)
            data_new=preprocess(data[i1:i1+w1,i2:i2+w2])
            data_new.shape
            output = run_prediction(data_new, model)
            logits[count1,count2] = output 
            inputted[count1,count2]=np.squeeze(data_new)
            count2+=1
        count1+=1
    return inputted, logits
    #return logits


[inputted,logits]=sliding_window(data)       
  
    

In [ ]:
#check output sizes
#print(inputted.shape)
print(logits.shape)

In [ ]:
np.save('/scratch/users/prgiven/continuous_data/continuous_processed/processed_data/day7/1a_logits.npy',logits)
np.save('/scratch/users/prgiven/continuous_data/continuous_processed/processed_data/day7/1a_windows.npy',inputted)
#np.save('/scratch/users/prgiven/continuous_data/continuous_processed/processed_data/20151024_091444/logits_second_half_20151024_091444_to_20151024_111444_chans2310_to_3334.npy',logits)
#np.save('/scratch/users/prgiven/continuous_data/continuous_processed/processed_data/20151024_091444/inputted_second_half_20151024_091444_to_20151024_111444_chans2310_to_3334.npy',inputted)

### Convert logits to probabilities

In [ ]:
 probability = tf.nn.sigmoid(logits).numpy()

In [ ]:
probability.shape

In [ ]:
print(probability[0])

### List High Probability Predictions (greater than 90% probability)

In [ ]:
print("High Probability Events",probability[probability>0.9])
print("High Probability Event Indices",np.where(probability>0.9))

### Total number of windows vs number of windows with predicted microseismic event with probability > 90%

In [ ]:
print("Total Windows",len(probability[probability>=0]))
print("Number of Positive Windows with <90% Probability",len(probability[probability>0.9]))

### Inputted windows predicted to have microseismic events (probability 90% or higher)

In [ ]:
positive_predictions=np.where(probability>0.9)
print(positive_predictions)
positive_windows=inputted[positive_predictions[0],positive_predictions[1],:,:]
print(positive_windows.shape)

In [ ]:
np.save('/scratch/users/prgiven/continuous_data/continuous_processed/processed_data/day8/1a_pos_winds.npy',positive_windows)

In [ ]:
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

plt.rcParams["figure.figsize"] = [10, 10]
plt.rcParams["figure.autolayout"] = True
fig, ax = plt.subplots()
plt.xlabel('Relative distance along array (m)')
plt.ylabel('Relative time (ms)')
plt.title('Envelope Movie')



ims = []
for i in range(len(probability[probability>0.9])):
    im = ax.imshow(abs((scipy.signal.hilbert2(positive_windows[i,:,:]))).T,cmap='rainbow',vmin=0,vmax=10, animated=True)
    ims.append([im])

ani = animation.ArtistAnimation(fig,ims,blit=True,interval=10)
#ani = FuncAnimation(fig,ims,frames=123,interval=10)
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 2**128
HTML(ani.to_jshtml())

In [ ]:
#change directory and desired saved file name 
f = "/scratch/users/prgiven/3a_last_day.gif" 
#writervideo = animation.FFMpegWriter(fps=1) 
ani.save(f, fps=1)